# ab-roller-rep-counter-machkevds

## Motivation Story & Introduction
As a child, I never had the opportunity of being confortable with my physical image, growing up in a family with lack of education in terms of healthy lifestyle habits, I was always considered heavy. After some resilientful times, I was able to shift reality into a more healthy lifestyle, achieving the opposite.

One of my favorite exercises, "Ab Wheels" or "Ab Rollers", as someone who was very overweight, and never had the opportunity of being "visually healthy", this exercise significantly helped with my core development.

This personal journey also brought up a technical curiosity. After encountering a LinkedIn post showcasing an AI-powered push-up counter built with similar libraries, this inspired me to explore how these computer vision and machin learning techniques could be applied to the ab wheel, into creating a real-time tracking system.

With these main motivations, my goal is to build tools that can better the health of individuals.


---

## Project Goal & Functionalities
This project aims to build a **real-time system** that accurately counts repetitions of the **kneeled ab wheel exercise** using a standard webcam feed. It serves as a robust prototype for an AI-powered fitness tracker, demonstrating advanced computer vision and state machine logic.

The prototype provides:
* **Real-time Pose Estimation:** Visualizes body landmarks during the exercise.
* **Accurate Repetition Counting:** Tracks and counts completed reps.
* **Robust Error Handling:** Distinguishes between full reps, partial reps, stalled movements, and lost poses.
* **Live Visual Feedback:** Displays hip angle, current rep count, and exercise state.
* **Session Recording:** Captures the entire annotated workout session into a video file for review.



---

## Key Features & Technical Highlights
* **Core Technology:** Leverages **MediaPipe Pose** for highly accurate real-time human pose estimation.
* **Smart State Machine:** Implements a sophisticated state machine (READY, EXTENDING, EXTENDED, RETRACTING, RECOVERY) to precisely track rep phases.
* **Hysteresis & Timeout Logic:** Utilizes distinct thresholds and time-based checks to prevent false counts from minor body movements, accidental stalls, or incomplete reps. Includes a dedicated `RECOVERY` state for explicit reset after failed attempts.
* **Video Processing:** Integrates `OpenCV` for frame manipulation and `FFmpeg` (via subprocess) for robust video recording and stitching of annotated frames.
* **Colab Environment:** Designed and optimized for execution within Google Colaboratory, supporting GPU acceleration for performance.


---

## How to Run This Project
To experience the AI-Powered Ab Wheel Repetition Counter yourself:

1.  **Open in Google Colab:**
    * Go to `colab.research.google.com`.
    * Click `File > Open notebook > GitHub`.
    * Paste your repository URL: `https://github.com/machkevds/ab-roller-rep-counter-machkevds`.
    * Select the main notebook file.
2.  **Set Up Runtime:**
    * Once the notebook loads, change the runtime type to `GPU` for optimal performance: `Runtime > Change runtime type > Hardware accelerator > GPU > Save`.
3.  **Grant Webcam Permissions:**
    * When you run the main code cell, your browser will prompt for webcam access. **You MUST grant this permission.**
4.  **Run the Notebook:**
    * Execute all cells sequentially. The primary execution is within the "Live Webcam Repetition Counter" cell.
    * The system will start capturing from your webcam, display the live processed feed, and record it to a video file.
5.  **User Instructions during Live Session:**
    * **Position yourself sideways** to the camera, ensuring your full body (from knees to head) is visible.
    * **Initial Setup:** Get into your kneeling ab wheel starting position and **hold it still for about 1-3 seconds** until the `State` overlay changes from `INITIAL_SETUP` to `READY`.
    * **Perform Reps:** Execute your ab wheel rollouts. The `Reps` count will increment, and the `State` will update.
    * **Test Edge Cases:** Deliberately try:
        * Stalling at the `EXTENDED` position for >3 seconds.
        * Performing a partial rollout.
        * Briefly moving out of frame.
    * **Stop the Session:** Click the square "Stop" button on the Colab cell to end the recording.
6.  **Download Your Recorded Session:**
    * After the session stops, a video file (`final_recorded_session.mp4`) will be generated. You can download it from the Colab file browser (folder icon on left sidebar).


---

## Technologies Used

* Python
* OpenCV (`cv2`)
* MediaPipe (`mediapipe`)
* NumPy
* FFmpeg (command-line tool)
* Google Colaboratory


---

## Code Walkthrough Showing How Project Works

1.  **Import Libraries**
    ```python
    import cv2
    import mediapipe as mp
    import numpy as np
    import os
    from google.colab.output import eval_js
    from IPython.display import display, Javascript, HTML
    from base64 import b64decode, b64encode
    import time
    import subprocess
    ```
    * **`cv2`**: OpenCV for image/video processing, drawing overlays.
    * **`mediapipe`**: Google's framework for advanced pose estimation.
    * **`numpy`**: For numerical operations on image data and angles.
    * **`os`**: For operating system tasks like creating temporary directories.
    * **`eval_js`, `display`, `Javascript`**: Colab-specific tools to bridge Python with the browser's webcam.
    * **`base64`**: For encoding/decoding image data transferred via JavaScript.
    * **`time`**: For timing operations and calculating FPS.
    * **`subprocess`**: To run external commands like FFmpeg.

2.  **Setup Webcam & Core Variables**
    ```python
    # Start webcam via JavaScript bridge
    start_webcam_stream(width=680, height=240)

    rep_count = 0
    current_state = 'INITIAL_SETUP' # Blocking state for clean start
    aborted_rep_flag = False      # Tracks if current rep attempt was valid
    TIMEOUT_SECONDS = 3           # Timeout for stalled movements
    INITIAL_SETUP_HOLD_SECONDS = 3 # Time to hold initial pose
    ```
    * **`start_webcam_stream`**: Initializes the webcam in your browser for live input.
    * **`rep_count`**: Stores total number of completed repetitions.
    * **`current_state`**: The core of the intelligent state machine, tracking the user's progress.
    * **`aborted_rep_flag`**: Crucial flag to prevent false counts after aborted reps.
    * **`TIMEOUT_SECONDS`**: Defines how long a user can stall in a rep phase before it's considered aborted.
    * **`INITIAL_SETUP_HOLD_SECONDS`**: Defines how long the user must hold the initial setup pose.

3.  **Live Stream Loop & Pose Processing**
    ```python
    with mp_pose.Pose(...) as pose:
        while processing_active:
            frame = capture_and_process_frame_from_webcam(quality=0.8)
            # ... error handling for frame ...
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(frame_rgb)
            annotated_frame = frame.copy()
            # ... rest of loop ...
    ```
    * **`while processing_active`**: The main loop continuously captures and processes frames.
    * **`capture_and_process_frame_from_webcam`**: Gets live image from the webcam.
    * **`pose.process(frame_rgb)`**: MediaPipe model analyzes the current frame to detect the human pose and get joint landmarks.

4.  **Angle Calculation & Blocking Setup Logic**
    ```python
    hip_angle = calculate_angle(r_shoulder_pix, r_hip_pix, r_knee_pix)

    if current_state == 'INITIAL_SETUP':
        if hip_angle > INITIAL_SETUP_MIN_ANGLE and hip_angle < INITIAL_SETUP_MAX_ANGLE:
            initial_setup_hold_counter += 1
            if initial_setup_hold_counter >= INITIAL_SETUP_HOLD_FRAMES:
                current_state = 'READY'
                aborted_rep_flag = False
                # ... other setup for first rep ...
        else:
            initial_setup_hold_counter = 0
    # ... else branch for other states ...
    ```
    * **`calculate_angle`**: Determines the critical hip angle from shoulder, hip, and knee landmarks.
    * **`INITIAL_SETUP`**: This is a **blocking gate**. The system waits here until the user holds a precise kneeling position (`INITIAL_SETUP_MIN/MAX_ANGLE`) for a defined duration. This prevents false counts from initial setup movements (which happened during testing).

5.  **Repetition Logic (State Machine)**
    ```python
    elif current_state == 'READY':
        if hip_angle > retracted_threshold:
            current_state = 'EXTENDING'
            aborted_rep_flag = False # New rep begins
    elif current_state == 'EXTENDING':
        if hip_angle > extended_threshold:
            current_state = 'EXTENDED'
        elif hip_angle < retracted_threshold:
            current_state = 'INITIAL_SETUP' # Partial rep detected, reset to setup
            aborted_rep_flag = True
    elif current_state == 'EXTENDED':
        if hip_angle < retracted_threshold:
            if not aborted_rep_flag: # CHECK: Only count if not aborted
                rep_count += 1
                # ... visual cue ...
            current_state = 'READY'
            aborted_rep_flag = False # Rep cycle ends, clear flag for next
    # ... other states: RETRACTING, RECOVERY ...
    ```
    * **State Transitions**: Hip angle changes trigger precise shifts between `READY`, `EXTENDING`, `EXTENDED`, `RETRACTING`.
    * **`aborted_rep_flag`**: Set to `True` if a rep is partially performed, stalls (`TIMEOUT_FRAMES`), or pose is lost. This ensures `rep_count` only increments for truly successful, non-aborted reps.
    * **`RECOVERY` State**: Handles situations where a rep is aborted mid-way or due to a timeout. It forces a complete reset to `INITIAL_SETUP` before any new reps can be tracked.

6.  **Visual Feedback & Frame Saving**
    ```python
    cv2.putText(annotated_frame, f'Reps: {rep_count}', ...)
    # ... other cv2.putText calls ...

    frame_filename = os.path.join(temp_frame_dir, f'frame_{frame_count:05d}.jpg')
    cv2.imwrite(frame_filename, annotated_frame)
    collected_frame_paths.append(frame_filename)
    # cv2_imshow(annotated_frame) # Commented for recording stability
    ```
    * **Visual Overlays**: Information like `Hip Angle`, `Reps`, `State` are drawn directly onto the frames. Rep completions trigger a visual flash.
    * **Frame Saving**: Each annotated frame is saved as an individual JPEG image to a temporary folder (`/tmp/recorded_frames`).

7.  **Final Video Stitching (FFmpeg)**
    ```python
    # In 'finally' block after loop ends
    final_output_fps = actual_processing_fps # Calculated from processed_frames_count / duration
    ffmpeg_cmd = [
        'ffmpeg', '-y', '-r', str(final_output_fps),
        '-i', os.path.join(temp_frame_dir, 'frame_%05d.jpg'),
        '-c:v', 'libx264', '-pix_fmt', 'yuv420p',
        '-vf', f"fps={final_output_fps}", output_final_video_filename
    ]
    subprocess.run(ffmpeg_cmd, ...)
    ```
    * **Robust Output**: After the live session, all saved individual frames are stitched together by the `ffmpeg` command-line tool. The video's playback FPS is set to match the average processing speed (`actual_processing_fps`), making sure the recorded video plays back at real-time speed. This method is for producing playable `.mp4` files.


---
## Future Enhancements (Optional Ideas)

* **Auditory Feedback:** Add sound cues (e.g., a "ding" on rep completion) when migrating to a local application.
* **Form Correction:** Implement logic to detect and provide feedback on common form errors (e.g. a check for contracting pelvis area as core is engaged to avoid back injuries).
* **User Interface:** Develop a standalone desktop or mobile application for a more integrated user experience.
* **Additional Exercises:** Extend the system to count other exercises (e.g., squats, push-ups).


---

---

In [ ]:
!pip install mediapipe opencv-python matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 20.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1

In [ ]:
# --- Live Webcam Repetition Counter ---

# Import necessary libraries
import cv2
import mediapipe as mp
import numpy as np
import os
from google.colab.patches import cv2_imshow # Imported, but usage in loop is commented for stability
from google.colab import files # For displaying/downloading saved video
from google.colab.output import eval_js # For executing JavaScript
from IPython.display import display, Javascript, HTML
from base64 import b64decode, b64encode
from base64 import b64decode, b64encode # Duplicate import, safe to remove one for cleanliness, but kept as requested
import time
import subprocess # For running FFmpeg command-line too

# Init MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# --- Function to Calculate Angle between three points (A, B, C) where B is the vertex.---
def calculate_angle(a, b, c):
  a = np.array(a)
  b = np.array(b)
  c = np.array(c)
  ba = a - b
  bc = c - b
  dot_product = np.dot(ba, bc)
  magnitude_ba = np.linalg.norm(ba)
  magnitude_bc = np.linalg.norm(bc)
  if magnitude_ba == 0 or magnitude_bc == 0:
      return 0.0
  cosine_angle = dot_product / (magnitude_ba * magnitude_bc)
  cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
  angle_radians = np.arccos(cosine_angle)
  angle_degrees = np.degrees(angle_radians)
  return angle_degrees

# --- JavaScript for Live Webcam Feed ---
def start_webcam_stream(quality=0.8, width=680, height=240):
  js = Javascript('''
      var video;
      var stream;
      var canvas;
      var ctx;
      var imageData;
      var imageElement;

      async function setupWebcam(width, height) {
          video = document.createElement('video');
          video.playsInline = true;
          video.autoplay = true;
          video.width = width;
          video.height = height;
          video.style.display = 'none'; //video element hidden

          document.body.appendChild(video);

          stream = await navigator.mediaDevices.getUserMedia({video: true});
          video.srcObject = stream;
          await video.play();

          // canvas to capture frames from stream
          canvas = document.createElement('canvas');
          canvas.width = video.videoWidth;
          canvas.height = video.videoHeight;
          ctx = canvas.getContext('2d');

          // image element to display current frame (cv2_imshow)
          imageElement = document.createElement('img');
          imageElement.style.display = 'block';
          imageElement.width = width;
          imageElement.height = height;
          document.body.appendChild(imageElement);

          google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      }

      //func to capture frame and return base 64 data
      function captureFrame(quality) {
          if (!video || !ctx || !canvas) {
              console.error("Webcam not set up.");
              return null;
          }
          ctx.drawImage(video, 0, 0, canvas.width, canvas.height);
          imageData = canvas.toDataURL('image/jpeg', quality);
          imageElement.src = imageData;
          return imageData;
      }

      //func to stop webcam and clean up elements
      function stopWebcam() {
          if (stream) {
              stream.getVideoTracks()[0].stop();
          }
          if (video) video.remove();
          if (canvas) canvas.remove();
          if (imageElement) imageElement.remove();
      }
  ''')
  display(js)
  #init webcam
  eval_js(f'setupWebcam({width}, {height})')
  print(f"Webcam stream started at {width}x{height}. Press the stop button or interrupt the kernel to stop.")

#func to capture a frame
def capture_and_process_frame_from_webcam(quality=0.8):
  data = eval_js(f'captureFrame({quality})')
  if data:
      binary = b64decode(data.split(',')[1])
      np_arr = np.frombuffer(binary, np.uint8)
      frame = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)
      return frame
  return None

def stop_webcam_stream_js():
  eval_js('stopWebcam()')
  print("Webcam stream stopped.")


# --- Main execution for Part 4 ---

# --- Repetition Counting Logic Variables ---
rep_count = 0

# new state that waits for user to be ready, this way unnecesary rep counts during setup are avoided
current_state = 'INITIAL_SETUP'
aborted_rep_flag = False

# Tuned thresholds
retracted_threshold = 105
extended_threshold = 170

# --- Visual Cue Variables ---
rep_flash_frames = 0
FLASH_DURATION = 5

# --- Timeout Variables ---
frame_count = 0
last_state_entry_frame = 0
TIMEOUT_SECONDS = 3
TIMEOUT_FRAMES = int(10 * TIMEOUT_SECONDS) # Adjust based on your actual stable FPS (e.g., if you get 7 FPS, use 7*3=21)

# new variables for Initial Setup Check
initial_setup_hold_counter = 0 # Counts frames where initial setup pose is held
INITIAL_SETUP_MIN_ANGLE = 20  # Min hip angle for a valid initial setup pose
INITIAL_SETUP_MAX_ANGLE = 50 # Max hip angle for a valid initial setup pose
INITIAL_SETUP_HOLD_SECONDS = 2 # How long (in seconds) to hold the initial pose
INITIAL_SETUP_HOLD_FRAMES = int(30 * INITIAL_SETUP_HOLD_SECONDS) # Convert seconds to frames


print("Setting up live webcam and recording...")

# Start JavaScript webcam stream
webcam_width = 680
webcam_height = 240

start_webcam_stream(width=webcam_width, height=webcam_height)

# --- Frame Collection Setup (Instead of VideoWriter) ---
temp_frame_dir = '/tmp/recorded_frames'
os.makedirs(temp_frame_dir, exist_ok=True)
collected_frame_paths = []

print(f"DEBUG: Frames will be saved to {temp_frame_dir}")

processing_active = True
start_time_processing = time.time()
processed_frames_count = 0


with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
  try:
      while processing_active:
          frame = capture_and_process_frame_from_webcam(quality=0.8)

          if frame is None:
              print("Failed to capture frame from webcam. Retrying...")
              time.sleep(0.1)
              continue

          frame_count += 1
          processed_frames_count += 1

          frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
          results = pose.process(frame_rgb)
          annotated_frame = frame.copy()

          # --- Repetition Counting and Display Logic ---
          try:
              if results.pose_landmarks:
                  mp_drawing.draw_landmarks(
                      annotated_frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                      landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
                  )

                  landmarks = results.pose_landmarks.landmark
                  r_shoulder_norm = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y]
                  r_hip_norm = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y]
                  r_knee_norm = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].y]

                  r_shoulder_pix = [int(r_shoulder_norm[0] * webcam_width), int(r_shoulder_norm[1] * webcam_height)]
                  r_hip_pix = [int(r_hip_norm[0] * webcam_width), int(r_hip_norm[1] * webcam_height)]
                  r_knee_pix = [int(r_knee_norm[0] * webcam_width), int(r_knee_norm[1] * webcam_height)]

                  hip_angle = calculate_angle(r_shoulder_pix, r_hip_pix, r_knee_pix)

                  # --- State Machine, update with INITIAL STATE to avoid misscounts during preparation ---

                  # new blocking logic If in INITIAL_SETUP, ONLY process its transition
                  if current_state == 'INITIAL_SETUP':
                      if hip_angle > INITIAL_SETUP_MIN_ANGLE and hip_angle < INITIAL_SETUP_MAX_ANGLE:
                          initial_setup_hold_counter += 1
                          if initial_setup_hold_counter >= INITIAL_SETUP_HOLD_FRAMES:
                              current_state = 'READY'
                              aborted_rep_flag = False
                              last_state_entry_frame = frame_count
                      else:
                          initial_setup_hold_counter = 0
                  else: # Only proceed rep counting states if NOT in INITIAL_SETUP

                      # check for timeout in current state
                      if (current_state == 'EXTENDING' or current_state == 'RETRACTING' or current_state == 'EXTENDED') and \
                         (frame_count - last_state_entry_frame > TIMEOUT_FRAMES):
                          if current_state == 'EXTENDED':
                              current_state = 'RECOVERY' # Timeout in EXTENDED is a failed rep -> RECOVERY
                              aborted_rep_flag = True # Mark attempt as aborted
                              # print(f"DEBUG: Timeout in EXTENDED phase. Transitioning to RECOVERY.")
                          else:
                              current_state = 'INITIAL_SETUP' # Back to INITIAL_SETUP on timeout in transition
                              aborted_rep_flag = True # Mark attempt as aborted
                              # print(f"DEBUG: Timeout in transition phase ({current_state}). Resetting to INITIAL_SETUP.")
                          last_state_entry_frame = frame_count

                      # ready state logic now goes after the INITIAL_SETUP
                      elif current_state == 'READY':
                          # transition to extending if angle is increasing past retracted_threshold
                          if hip_angle > retracted_threshold:
                              current_state = 'EXTENDING'
                              last_state_entry_frame = frame_count
                              aborted_rep_flag = False # Starting fresh rep, clear any previous abort flag

                      elif current_state == 'EXTENDING':
                          # if fully extended
                          if hip_angle > extended_threshold:
                              current_state = 'EXTENDED'
                              last_state_entry_frame = frame_count
                          #if user retracts before full extension (partial reo)
                          elif hip_angle < retracted_threshold:
                              current_state = 'INITIAL_SETUP' # Revert to INITIAL_SETUP on partial rep
                              aborted_rep_flag = True # Mark this attempt as aborted
                              last_state_entry_frame = frame_count

                      elif current_state == 'EXTENDED':
                          # if retracting from extended
                          if hip_angle < retracted_threshold:
                              #count is rep was not aborted
                              if not aborted_rep_flag: # CRITICAL FIX CHECK HERE
                                  rep_count += 1
                                  rep_flash_frames = FLASH_DURATION
                              # print(f"DEBUG: Rep completed! New count: {rep_count}. Transitioned to READY (Angle: {int(hip_angle)})")
                              current_state = 'READY' # ready for next rep
                              aborted_rep_flag = False # Rep completed (or skipped if aborted), clear flag
                              last_state_entry_frame = frame_count


                      elif current_state == 'RETRACTING':
                          # If retracted
                          if hip_angle < retracted_threshold:
                              current_state = 'READY'
                              last_state_entry_frame = frame_count
                          # If they re-extend before full retraction (bounce or re-attempt)
                          elif hip_angle > extended_threshold:
                              current_state = 'EXTENDING'
                              last_state_entry_frame = frame_count

                      elif current_state == 'RECOVERY':
                          # Only way out of RECOVERY is to return to a READY (bent) position
                          if hip_angle < retracted_threshold:
                              current_state = 'INITIAL_SETUP' # Revert to INITIAL_SETUP from RECOVERY
                              last_state_entry_frame = frame_count
                          # else, stay in RECOVERY until proper reset

                  # --- Display Logic ---
                  rep_color = (0, 0, 255) # Red
                  if rep_flash_frames > 0:
                      rep_color = (0, 255, 0) # Green
                      rep_flash_frames -= 1

                  cv2.putText(annotated_frame, f'Hip Angle: {int(hip_angle)}',
                              (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                  cv2.putText(annotated_frame, f'Reps: {rep_count}',
                              (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, rep_color, 2, cv2.LINE_AA)
                  cv2.putText(annotated_frame, f'State: {current_state}',
                              (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)


                  # message for RECOVERY state
                  if current_state == 'RECOVERY' or current_state == 'INITIAL_SETUP': # Display message for SETUP/RECOVERY states
                       cv2.putText(annotated_frame, "SETUP/RESET: Return to Start Position",
                                  (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)
              else:
                  cv2.putText(annotated_frame, "No pose detected!",
                              (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                  current_state = 'INITIAL_SETUP' # If pose is lost, go to INITIAL_SETUP
                  aborted_rep_flag = True

          except Exception as e:
              cv2.putText(annotated_frame, f"Error processing landmarks: {str(e)[:50]}...",
                          (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2, cv2.LINE_AA)
              current_state = 'INITIAL_SETUP' # On error, go to INITIAL_SETUP
              aborted_rep_flag = True

          # --- Save frame as image file ---
          frame_filename = os.path.join(temp_frame_dir, f'frame_{frame_count:05d}.jpg')
          # Check if imwrite is successful
          imwrite_success = cv2.imwrite(frame_filename, annotated_frame)
          if imwrite_success:
              collected_frame_paths.append(frame_filename)
          else:
              print(f"WARNING: Failed to write frame {frame_filename} to disk. Frame might be empty or corrupted.")

          #cv2_imshow(annotated_frame) # useful to mirror while testing for cases


  except KeyboardInterrupt:
      print("\nProcessing interrupted by user. Stopping webcam and saving video...")
  except Exception as e:
      print(f"\nAn unexpected error occurred during processing: {e}")
  finally:
      # webcam and video writer are released
      stop_webcam_stream_js()

      end_time_processing = time.time()
      duration_processing = end_time_processing - start_time_processing
      actual_processing_fps = processed_frames_count / duration_processing if duration_processing > 0 else 0

      print(f"DEBUG: Total processed frames: {processed_frames_count}, Total duration: {duration_processing:.2f}s, Actual Avg Processing FPS: {actual_processing_fps:.2f}")

      # --- FFmpeg Stitching ---
      output_final_video_filename = 'final_recorded_session.mp4'
      if len(collected_frame_paths) > 0:
          print(f"Stitching {len(collected_frame_paths)} frames into video using FFmpeg...")
          final_output_fps = actual_processing_fps if actual_processing_fps > 0 else 30.0
          if final_output_fps > 60.0:
              final_output_fps = 60.0 # to 60fps for playback smoothness

          ffmpeg_cmd = [
              'ffmpeg',
              '-y',  # Overwrite output file without asking
              '-r', str(final_output_fps), # Input frame rate for stitching
              '-i', os.path.join(temp_frame_dir, 'frame_%05d.jpg'),
              '-c:v', 'libx264', # H.264 codec
              '-pix_fmt', 'yuv420p', # Pixel format for broad compatibility
              '-vf', f"fps={final_output_fps}", # output video FPS
              output_final_video_filename
          ]
          try:
              subprocess.run(ffmpeg_cmd, check=True, capture_output=True, text=True)
              print(f"FFmpeg stitching complete. Video saved as: {output_final_video_filename}")
          except subprocess.CalledProcessError as e:
              print(f"Error during FFmpeg stitching:\n{e.stdout}\n{e.stderr}")
              print("FFmpeg command failed. Check if frames were saved correctly.")
          except FileNotFoundError:
              print("FFmpeg command not found. It should be pre-installed in Colab.")
      else:
          print("No frames were collected to stitch into a video.")

      # clean up temporary frame directory
      try:
          if os.path.exists(temp_frame_dir):
              import shutil
              shutil.rmtree(temp_frame_dir)
              print(f"Cleaned up temporary frame directory: {temp_frame_dir}")
      except Exception as e:
          print(f"Error cleaning up temporary directory: {e}")

      print("Live webcam processing and recording finished.")


# --- Displaying Saved Recorded Video in Colab (for verification) ---
def show_local_mp4_as_html(file_name):
  html = ''
  try:
      video = open(file_name,'rb').read()
      src = 'data:video/mp4;base64,' + b64encode(video).decode()
      html += '<video width=800 controls autoplay><source src="%s" type="video/mp4"></video>' % src
  except FileNotFoundError:
      html += f"<p>Error: Video file '{file_name}' not found.</p>"
  except Exception as e:
      html += f"<p>Error displaying video: {e}</p>"
  return HTML(html)

print(f"\nAttempting to display recorded session: {output_final_video_filename}")
if os.path.exists(output_final_video_filename):
  display(show_local_mp4_as_html(output_final_video_filename))
else:
  print(f"Recorded video {output_final_video_filename} not found. Ensure recording was successful.")



